加载已经训练好的三组超参对应的网络参数，直接画图。调整画图细节

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from itertools import cycle

sns.set_style('whitegrid')

In [ ]:
# ===================================================================
# 单元格 2: 加载已保存的预测数据
# ===================================================================

json_path = "/kaggle/input/champion-test-datasets/champion_test_predictions.json"

all_results = {} # 初始化一个新字典来存储NumPy格式的数据

try:
    with open(json_path, 'r') as f:
        # 1. 从JSON文件加载数据 (所有值都是Python列表)
        loaded_data_from_json = json.load(f)
    
    # 2. 将列表转换回 NumPy 数组
    for name, data in loaded_data_from_json.items():
        all_results[name] = {
            "y_true": np.array(data["y_true"]),
            "y_pred_labels": np.array(data["y_pred_labels"]),
            "y_prob_scores": np.array(data["y_prob_scores"]),
            "y_true_binarized": np.array(data["y_true_binarized"]),
            "n_classes": data["n_classes"],
            "class_names": data["class_names"]
        }
        
    print(f"成功从JSON文件加载并重建了 {len(all_results)} 组预测数据！")
    print(f"   已加载的冠军: {list(all_results.keys())}")

except FileNotFoundError:
    print(f"❌ 错误: 文件未在路径 {json_path} 找到。")
    print("   请确保已上传JSON文件并正确修改了 'json_path' 变量。")
except Exception as e:
    print(f"❌ 加载或转换数据时出错: {e}")

In [ ]:
# ===================================================================
#  绘制混淆矩阵
# ===================================================================
from matplotlib import rcParams 
# 设置全局字体配置 
# 这将影响此单元格中创建的所有图表
rcParams['font.weight'] = 'bold'         
rcParams['axes.titleweight'] = 'bold'    
rcParams['axes.labelweight'] = 'bold'   
rcParams['figure.titleweight'] = 'bold'  

if 'all_results' in locals() and all_results:
    print("--- 正在绘制 混淆矩阵 (Confusion Matrices) ---")
    
    num_champions = len(all_results)
    fig, axes = plt.subplots(nrows=1, ncols=num_champions, figsize=(8 * num_champions, 7))
    
    if num_champions == 1: axes = [axes]
    
    for ax, (name, results) in zip(axes, all_results.items()):
        cm = confusion_matrix(results["y_true"], results["y_pred_labels"])
        class_names = results["class_names"]
        
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, yticklabels=class_names, ax=ax,
                    annot_kws={"size": 14}) 
        
        ax.set_xlabel('Predicted Label', fontsize=20)
        ax.set_ylabel('True Label', fontsize=20)
      
        ax.set_yticklabels(ax.get_yticklabels(), rotation=45, va='center')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    plt.subplots_adjust(wspace=0.3)
    
    output_filename = 'champion_confusion_matrices.pdf'
    plt.savefig(output_filename, format='pdf',bbox_inches='tight')
    print(f"\n[INFO] ✅ 混淆矩阵图已成功保存为: {output_filename}")
    
    plt.show() 

    for name, results in all_results.items():
        print("\n" + "="*50)
        print(f"分类报告 (Classification Report) for {name}:")
        print(classification_report(results["y_true"], results["y_pred_labels"], 
                                    target_names=results["class_names"]))
        print("="*50)
else:
    print("错误: 'all_results' 变量未找到")

In [ ]:
# ===================================================================
# 绘制ROC曲线 (带局部放大功能)
# ===================================================================
from matplotlib import rcParams 
from matplotlib.patches import Rectangle, ConnectionPatch
rcParams['font.weight'] = 'bold'        
rcParams['axes.titleweight'] = 'bold'    
rcParams['axes.labelweight'] = 'bold'    
rcParams['figure.titleweight'] = 'bold'  

if 'all_results' in locals() and all_results:
    print("绘制 多分类ROC曲线 (Micro & Macro Averages)")
    
    fig, (ax_micro, ax_macro) = plt.subplots(nrows=1, ncols=2, figsize=(20, 9))
    
    colors = {"Human Baseline": "C0", "Random Search Champion": "C1", "BO Champion": "C2"}

    # 创建局部放大图 (Inset Axes) ---
    # 语法: [left, bottom, width, height] (比例)
    ax_micro_inset = ax_micro.inset_axes([0.4, 0.1, 0.58, 0.58])
    ax_macro_inset = ax_macro.inset_axes([0.4, 0.1, 0.58, 0.58])
    # -----------------------------------------------

    for name, results in all_results.items():
        y_true_bin = results["y_true_binarized"]
        y_prob = results["y_prob_scores"]
        n_classes = results["n_classes"]
        color = colors.get(name, 'black')

        # Micro-average ROC
        fpr_micro, tpr_micro, _ = roc_curve(y_true_bin.ravel(), y_prob.ravel())
        auc_micro = auc(fpr_micro, tpr_micro)
        # 在主图上绘制
        ax_micro.plot(fpr_micro, tpr_micro, lw=2.5, color=color,
                      label=f'{name} (Micro-avg AUC = {auc_micro:0.3f})')
        # 在放大图上绘制
        ax_micro_inset.plot(fpr_micro, tpr_micro, lw=2.5, color=color)
        
        # 2. 计算 Macro-average ROC
        all_fpr = {}
        all_tpr = {}
        for i in range(n_classes):
            all_fpr[i], all_tpr[i], _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
        
        all_fpr_grid = np.unique(np.concatenate([all_fpr[i] for i in range(n_classes)]))
        mean_tpr = np.zeros_like(all_fpr_grid)
        for i in range(n_classes):
            mean_tpr += np.interp(all_fpr_grid, all_fpr[i], all_tpr[i])
        mean_tpr /= n_classes
        
        auc_macro = auc(all_fpr_grid, mean_tpr)
        # 在主图上绘制
        ax_macro.plot(all_fpr_grid, mean_tpr, lw=2.5, color=color,
                      label=f'{name} (Macro-avg AUC = {auc_macro:0.3f})')
        #在放大图上绘制
        ax_macro_inset.plot(all_fpr_grid, mean_tpr, lw=2.5, color=color)

    # 3. 格式化 Micro-average 主图
    ax_micro.plot([0, 1], [0, 1], 'k--', lw=1)
    ax_micro.set_xlim([0.0, 1.0]); ax_micro.set_ylim([0.0, 1.05])
    ax_micro.set_xlabel('False Positive Rate', fontsize=20)
    ax_micro.set_ylabel('True Positive Rate', fontsize=20)
    # ax_micro.set_title('Multi-class ROC: Micro-Average', fontsize=16)
    ax_micro.legend(loc="lower right", fontsize=12, frameon=True)
    ax_micro.grid(True)

    ax_micro_inset.set_xlim(0.0, 0.05) 
    ax_micro_inset.set_ylim(0.95, 1.005) 
    ax_micro_inset.grid(True, linestyle=':', alpha=0.7)
    # ax_micro_inset.set_title('Zoom In (Top-Left Corner)', fontsize=10)
    # 绘制连接线
    ax_micro.indicate_inset_zoom(ax_micro_inset, edgecolor="black", linewidth=1.5)
    
    
    ax_macro.plot([0, 1], [0, 1], 'k--', lw=1)
    ax_macro.set_xlim([0.0, 1.0]); ax_macro.set_ylim([0.0, 1.05])
    ax_macro.set_xlabel('False Positive Rate', fontsize=20)
    ax_macro.set_ylabel('True Positive Rate', fontsize=20)
    # ax_macro.set_title('Multi-class ROC: Macro-Average', fontsize=16)
    ax_macro.legend(loc="lower right", fontsize=12, frameon=True)
    ax_macro.grid(True)

   
    ax_macro_inset.set_xlim(0.0, 0.05) 
    ax_macro_inset.set_ylim(0.95, 1.005) 
    ax_macro_inset.grid(True, linestyle=':', alpha=0.7)
    # ax_macro_inset.set_title('Zoom In (Top-Left Corner)', fontsize=10)
    # 绘制连接线
    ax_macro.indicate_inset_zoom(ax_macro_inset, edgecolor="black", linewidth=1.5)

    output_filename = 'champion_roc_curves.pdf'
    plt.savefig(output_filename, format='pdf',bbox_inches='tight')
    print(f"\n[INFO] ✅ ROC曲线图已成功保存为: {output_filename}")
    
    plt.show()

else:
    print("错误: 'all_results' 变量未找到")